# Create one dataset for each profiler

- get profiler data organized by each serving configuration
- run {script}.py to go trough all experiment runs

- Input:
  - results_*/{profiler data}.csv
- Output
  - results_*/processed/{profiler}_processed.csv

[nov] 
- run before 01_get_info_wattmeter.py 01_get_info_energi.py 01_get_info_nvidia.py
- This uses processed/ and join them, different m_1/results_1/processed, m_2/results_1/processed .... in m_1/results_1/processed


In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [50]:
SAVE_TABLES = True
#SAVE_FIGS = False

In [64]:
#device = "gpu" 
# all_dir/results_*
#all_dir = f"D:/GAISSA/energy-repo/last_repo/june_{device}_test/" # 
#new_results = f"D:/GAISSA/energy-repo/last_repo/{device}_june_phi2_01/" # 

# D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/
# D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_01/
#all_dir = f"D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/" # change
#new_results = f"D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_03/" # change if new dataset

all_dir = f"D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/nov_cpuep_00/" # change
new_results = f"D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/nov_cpuep_05/" # change if new dataset

In [59]:
def change_model(old,new, model):
    try:
        new = new[new['runtime'] != 'idle']
    
        old = old[old['model'] != model]
    except:
        print("check dfs")
    
    return pd.concat([old, new], )#ignore_index=True

In [60]:
def add_models(old,new):
    
    try:
        new = new[new['runtime'] != 'idle']
    
        #old = old[old['model'] != model]
    except:
        print("check dfs")
    
    return pd.concat([old, new], )#ignore_index=True

## energibridge files

In [65]:
os.chdir(all_dir)
for i in range(1,11):
    results_dir = f'results_{i}/' 
    print(f"++++++++--------------------results_dir: {results_dir}---------------------------++++++++:")
    
    #print(f"results_dir: {results_dir}")
    processed_dir = all_dir + results_dir +"processed/" # 

    os.chdir(all_dir)

    csv_dir = results_dir

    # after running 01_get_info_energibridge.py
    profilers_prefixes = ['energi', 'nvidia','wattmeter']
    prefix = profilers_prefixes[0] # change if other profiler
    file = f'processed/{prefix}_processed_updated.csv' # CHANGE if new dataset, first: energi_processed, then energi_processed_updated
    combined_df = pd.read_csv(os.path.join(csv_dir, file))
    print(combined_df[:1])
    print("---- last values in first dataset:")
    print(combined_df[-1:])

    print("--------------------new results---------------------------:")
    os.chdir(new_results)
    # after running 01_get_info_energibridge.py
    file = f'processed/{prefix}_processed.csv' # same, from new model to be added
    new_df = pd.read_csv(os.path.join(csv_dir, file))
    print(new_df[:1])
    print("---- last values in second dataset:")
    print(new_df[-1:])

    print(len(combined_df))
    print(len(new_df))
    original_df = combined_df.copy()
    combined_df = add_models(combined_df, new_df,)
    print(len(combined_df))

    print(f"unique models: {combined_df['model'].unique()}")
    #combined_df
    # Sort the DataFrame by the 'timestamp' column
    processed_updated_name = f"{prefix}_processed_updated.csv"
    if SAVE_TABLES:
        combined_df.to_csv(processed_dir+processed_updated_name, index=False)
        print(f"Results saved in {processed_dir+processed_updated_name}")
    else:
        print(f"Results NOT saved in {processed_dir+processed_updated_name}")

    #verify timestamp
    print("--------------------verify timestamps match---------------------------:")
    os.chdir(all_dir)
    processed_original = pd.read_csv(results_dir + 'processed/time_marks.csv', )

    print("from processed_original:")
    print(processed_original['time'].iloc[0])
    print(processed_original['time'].iloc[len(processed_original)-1])
    print("Original ")
    print(original_df['time'].iloc[0])
    print(original_df['time'].iloc[len(original_df)-1])
    
    print("new added")
    print(new_df['time'].iloc[0])
    print(new_df['time'].iloc[len(new_df)-1])
    
    print(f"{prefix} updated")
    print(combined_df['time'].iloc[0])
    print(combined_df['time'].iloc[len(combined_df)-1])

++++++++--------------------results_dir: results_1/---------------------------++++++++:
                      time runtime model  CPU_ENERGY (J)  TOTAL_MEMORY  \
0  2024-06-12 02:19:20.818    idle  idle   849107.627319   66575949824   

   TOTAL_SWAP  USED_MEMORY  USED_SWAP  AVG_CPU_USAGE_SAMP      label  \
0  2147479552   2730065920          0            6.485433  idle_idle   

   avg_TOTAL_MEMORY  avg_TOTAL_SWAP  avg_USED_MEMORY  avg_USED_SWAP  \
0      6.657595e+10    2.147480e+09     3.243619e+09            0.0   

   avg_cpu_usage_config       energy  avg_used_memory_pct_config  
0              0.147192  5513.152359                    4.872058  
---- last values in first dataset:
                           time runtime       model  CPU_ENERGY (J)  \
263388  2024-11-20 01:20:08.440      ov  bloomz-1b1    3.796407e+07   

        TOTAL_MEMORY  TOTAL_SWAP  USED_MEMORY  USED_SWAP  AVG_CPU_USAGE_SAMP  \
263388   66582405120  2147479552   8454893568   80183296           32.638375   

  

In [7]:
# [nov] verify time_marks match with energi_processed


# combined_df['time'] = pd.to_datetime(combined_df['Time'],unit='ms')
# combined_df['time'] += pd.Timedelta(hours=2)

# new_df['time'] = pd.to_datetime(new_df['Time'],unit='ms')
# new_df['time'] += pd.Timedelta(hours=1) 


In [8]:
#combined_df[combined_df['model'] == 'tiny']

In [9]:
#combined_df[:10]